<a href="https://colab.research.google.com/github/flora0110/OpenCSR_RL/blob/main/F2FSpareS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import pandas as pd
from scipy.sparse import dok_matrix

In [5]:
# read GenericsKB to Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Research/GenericsKB-Best.tsv',sep='\t')

In [6]:
df[:10]

,SOURCE,TERM,QUANTIFIER,GENERIC SENTENCE,SCORE
0,Waterloo,aa battery,NaN,AA batteries maintain the settings if the powe...,0.350923
1,ARC,aardvark female,NaN,Aardvark females appear to come into season on...,0.570737
2,ARC,aardvark hole,NaN,Aardvark holes are used by small buck as a res...,0.574909
3,Waterloo,aardvark skin,NaN,Aardvark skin is thick and sparsely haired.,0.444273
4,WordNet3.0,aardvark,NaN,Aardvark isa mammal.,1.000000
5,ARC,aardvark,NaN,Aardvarks also dig to get food.,0.590054
6,ARC,aardvark,NaN,Aardvarks also eat locusts and a type of grass...,0.724097
7,ARC,aardvark,NaN,"Aardvarks also require sandy soil, as opposed ...",0.708718
8,ARC,aardvark,NaN,Aardvarks are a nocturnal creature.,0.712522
9,ARC,aardvark,NaN,Aardvarks are about the size of a small pig.,0.620401


## make Sparse Fact-to-Fact Index S

In [7]:
# get unique concept
# use set becuase we only care if the element exsit, so we dont need index
# set not allow duplicate element
concepts = set(df["TERM"].unique())

In [8]:
concepts_list = list(concepts)
concepts_list[:10]

['fine florist',
 nan,
 'honey comb',
 'final',
 'animal language',
 'seismic migration',
 'corticosteroid treatment',
 'cyan',
 'unsubstituted amide',
 'fewer bone']

In [9]:
# create a concept2idx(dic)，let every concept has index
concept2idx = {c: i for i, c in enumerate(concepts)}

In [12]:
#---- tes code----

from itertools import islice

# print first 10 element
for key, value in islice(concept2idx.items(), 10):
    print(key, value)


fine florist 0
nan 1
honey comb 2
final 3
animal language 4
seismic migration 5
corticosteroid treatment 6
cyan 7
unsubstituted amide 8
fewer bone 9


In [13]:
# create a spare matrix S，size=|F| × |F|
S = dok_matrix((len(concepts), len(concepts)), dtype=bool)

### convert "GENERIC SENTENCE" to list of concept

In [80]:
how_many_sentence_we_use = 10000

In [26]:
import nltk
from nltk.stem import WordNetLemmatizer

In [30]:
# download necessary NLTK data (only need to do this once)
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("words")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [38]:
df.shape

(1020868, 5)

In [83]:
short_df = df.sample(n=how_many_sentence_we_use, random_state=42).reset_index(drop=True)
short_df.shape

(10000, 5)

In [104]:
facts_GENERICSENTENCE = short_df['GENERIC SENTENCE']

In [105]:
facts_GENERICSENTENCE[:10]

0    Hawks start breeding when they are one year old .
1                           Companies have candidates.
2                          Most male owls bring meals.
3    Colon cancer is now the leading cause of cance...
4    Some objects have sharp geometric features suc...
5    Citizenship is in many ways a difficult and pe...
6    Some lizards change color in response to their...
7                          Every computer has a clock.
8                          All glands receive vessels.
9    Alcohol affects the central nervous system of ...
Name: GENERIC SENTENCE, dtype: object

In [84]:
concepts_GENERICSENTENCE = []
# 定義lemmatizer
lemmatizer = WordNetLemmatizer()

for idx, row in short_df.iterrows():
  #print(str(idx)+" : "+row["GENERIC SENTENCE"])
  tokens = nltk.word_tokenize(row["GENERIC SENTENCE"])
  #print(tokens)
  # perform part-of-speech tagging to identify nouns and noun phrases
  pos_tags = nltk.pos_tag(tokens)
  #print(pos_tags)
  concepts = set()
  for i in range(len(pos_tags)):
    if pos_tags[i][1].startswith("NN"):  # check if the word is a noun
      # 使用lemmatizer將單數名詞還原為原形
      lemma = lemmatizer.lemmatize(pos_tags[i][0], pos=pos_tags[i][1][0].lower())
      # 加入到概念中
      concepts.add(lemma)
  #print(concepts)
  #print("\n")
  concepts_GENERICSENTENCE.append(concepts)
    

In [85]:
concepts_GENERICSENTENCE

[{'Hawks', 'year'},
 {'Companies', 'candidate'},
 {'bring', 'meal', 'owl'},
 {'Colon', 'cancer', 'cause', 'death', 'nonsmoker'},
 {'corner', 'crease', 'feature', 'object'},
 {'Citizenship', 'life', 'way'},
 {'color', 'lizard', 'mood', 'response'},
 {'clock', 'computer'},
 {'gland', 'vessel'},
 {'Alcohol', 'driver', 'system'},
 {'Silk', 'resistant', 'water'},
 {'energy', 'plant'},
 {'Slugs', 'marigold'},
 {'cache', 'space'},
 {'computer', 'graphic', 'picture'},
 {'centrifugation', 'measure', 'rate', 'sediment'},
 {'Protection', 'detection'},
 {'Hate', 'community', 'crime', 'fabric', 'society'},
 {'Communication', 'basis', 'decision'},
 {'Geomancies', 'divination'},
 {'baby', 'developing', 'nutrition'},
 {'Pressure', 'pressure'},
 {'birth', 'boar', 'dog', 'drainage', 'man', 'remain', 'system'},
 {'Packers', 'cell', 'part'},
 {'compound', 'room', 'substance', 'temperature'},
 {'Ear', 'microscopic', 'mite', 'organism', 'tick'},
 {'Ethics', 'practice', 'structure', 'theory'},
 {'Bats', 'day

In [86]:
type(concepts_GENERICSENTENCE)

list

In [110]:
facts_GENERICSENTENCE = facts_GENERICSENTENCE.tolist()

In [111]:
type(facts_GENERICSENTENCE)

list

#### 把concepts_GENERICSENTENCE先存起來

In [87]:
import pickle


# Open a file in binary mode
with open('concepts_GENERICSENTENCE_how_many_sentence_we_use.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(concepts_GENERICSENTENCE, f)


In [88]:
"""
import pickle

# Open the file in binary mode
with open('concepts_GENERICSENTENCE_how_many_sentence_we_use.pkl', 'rb') as f:
    # Use pickle to load the list from the file
    concepts_GENERICSENTENCE = pickle.load(f)

# The list can now be used in the program
print(concepts_GENERICSENTENCE)
"""

"\nimport pickle\n\n# Open the file in binary mode\nwith open('concepts_GENERICSENTENCE_how_many_sentence_we_use.pkl', 'rb') as f:\n    # Use pickle to load the list from the file\n    concepts_GENERICSENTENCE = pickle.load(f)\n\n# The list can now be used in the program\nprint(concepts_GENERICSENTENCE)\n"

### 利用concepts建立fact to fact的關係

In [114]:
import numpy as np
from scipy.sparse import lil_matrix

In [115]:
# 創建一個空的稀疏矩陣
num_facts = len(facts_GENERICSENTENCE)
S = lil_matrix((num_facts, num_facts), dtype=bool)

In [116]:
print(len(facts_GENERICSENTENCE))
print(num_facts)
print(type(S))

10000
10000
<class 'scipy.sparse._lil.lil_matrix'>


**連接規則**
if we can create a link fi → fj (i.e.,
Sij = 1) as follows:
- i != j
- |I| >= 1 
  - where I is the set of concepts that are mentioned in both fi and fj . Note that we remove the most frequent 100 concepts (e.g.,
human) from I.
- |I| < |fi|
  - We do not create links when all
concepts in fi are mentioned in fj , which are
usually redundant.
- |fj | − |I| >= 2. 
  - We create links only when there are more than two unseen concepts in fj which are not in fi

#### create a list with most frequent 100 concepts

In [91]:
how_many_cocept_too_common = 50

In [92]:
# create a list with most frequent 100 concepts
from collections import defaultdict
from nltk.tokenize import word_tokenize

In [93]:
# Step 1: Create a dictionary to store the frequency of each concept
freq_dict = defaultdict(int)

# Step 2-3: Tokenize each sentence and count the frequency of each concept
for sentence in concepts_GENERICSENTENCE:
  for concept in sentence:
      freq_dict[concept] += 1


In [94]:
# test code
first_10_items = list(freq_dict.items())[:10]

print(first_10_items)

[('Hawks', 4), ('year', 76), ('Companies', 9), ('candidate', 2), ('bring', 1), ('meal', 15), ('owl', 4), ('cancer', 71), ('nonsmoker', 1), ('cause', 65)]


In [95]:
# Step 4: Sort the dictionary by value in descending order
sorted_freq_dict = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)

In [98]:
# Step 5: Extract the top 100 concepts
most_common_concepts = [x[0] for x in sorted_freq_dict[:how_many_cocept_too_common]]

In [99]:
most_common_concepts

['part',
 'people',
 'water',
 'plant',
 'animal',
 'food',
 'life',
 'cell',
 'body',
 'child',
 'energy',
 'time',
 'disease',
 'system',
 'woman',
 'way',
 'form',
 'area',
 'tree',
 'world',
 'problem',
 'year',
 'growth',
 'cancer',
 'effect',
 'soil',
 'specie',
 'process',
 'bird',
 'cause',
 'source',
 'color',
 'health',
 'air',
 'insect',
 'material',
 'Water',
 'number',
 'group',
 'People',
 'family',
 'leaf',
 'development',
 'seed',
 'death',
 'environment',
 'thing',
 'blood',
 'organism',
 'light']

#### connect facts -> Sparse Fact-to-Fact Index S

In [100]:
# 定義連接規則
def connection_rules(concepts1, concepts2):
    common_concepts = concepts1.intersection(concepts2)
    common_concepts.difference_update(most_common_concepts)
    new_concepts = concepts2 - concepts1
    if(len(common_concepts) <= 0):
      return False
    elif(len(new_concepts) < 2):
      return False
    return len(common_concepts) > 0 and len(new_concepts) >= 2

In [117]:
# 對所有事實建立稀疏索引
for i in range(len(facts_GENERICSENTENCE)):
    for j in range(i+1, len(facts_GENERICSENTENCE)):
        if connection_rules(concepts_GENERICSENTENCE[i], concepts_GENERICSENTENCE[j]):
            S[i, j] = True
            S[j, i] = True  # 因為S是對稱矩陣，所以需要同時更新對稱元素

##### 存起來

In [119]:
from scipy.sparse import save_npz

# 存成 .npz 檔案
saving_usage_S = S.tocsr()
save_npz('S.npz', saving_usage_S)


In [ ]:
#from scipy.sparse import load_npz

#load_out_S = load_npz('S.npz')


## sparse retrieval

In [122]:
def fact_to_concept(fact):
  tokens = nltk.word_tokenize(fact)
  #print(tokens)
  # perform part-of-speech tagging to identify nouns and noun phrases
  pos_tags = nltk.pos_tag(tokens)
  #print(pos_tags)
  concepts = set()
  for i in range(len(pos_tags)):
    if pos_tags[i][1].startswith("NN"):  # check if the word is a noun
      # 使用lemmatizer將單數名詞還原為原形
      lemma = lemmatizer.lemmatize(pos_tags[i][0], pos=pos_tags[i][1][0].lower())
      concepts.add(lemma)
  return concepts

In [ ]:
def sparse_retrieval(question, kb, concept2idx, S):
    # 将问题分解为概念列表
    concepts = fact_to_concept(question)
    concept_idxs = [concept2idx[c] for c in concepts if c in concept2idx]
    
    # 对于每个包含至少一个概念的事实，计算与问题的相似度得分
    scores = np.zeros(len(kb))
    for i, row in kb.iterrows():
        # 跳过缺失的事实
        if pd.isnull(row['GENERIC SENTENCE']):
            continue
        # 计算事实和问题之间的共同概念数量
        fact_concepts = set(row['GENERIC SENTENCE'].split(' '))
        common_concepts = fact_concepts.intersection(set(concepts))
        # 如果事实中包含至少一个概念，则计算其与问题的相似度得分
        if len(common_concepts) > 0:
            j = concept_idxs[0]  # 取第一个概念的索引作为行索引
            k = concept2idx[row['TERM']]  # 取事实中的TERM概念的索引作为列索引
            # 在稀疏张量 S 中查找行索引 j 和列索引 k，如果存在则将得分加入相似度得分
            if j in S and k in S[j].indices:
                scores[i] = S[j][k]
    
    # 根据得分从高到低对事实进行排序并返回
    sorted_indices = np.argsort(scores)[::-1]
    return kb.iloc[sorted_indices]

In [ ]:
print(df.columns)

Index(['SOURCE', 'TERM', 'QUANTIFIER', 'GENERIC SENTENCE', 'SCORE'], dtype='object')


In [ ]:
sparse_retrieval("what can help alleviate global warming?",df,concept2idx,S)

ValueError: ignored